# co-training

In [1]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torchvision.models import resnet50

In [2]:
from math import floor
import numpy as np
import copy
import random
import pickle
import os
from pprint import pprint

In [3]:
def get_topk_predictions(pred, k):
    prob, label = torch.max(pred, 1)
    idx = torch.argsort(prob, descending=True)[:k]
    return prob[idx], label[idx], idx

In [4]:
# TODO extract some of the logic into another function (so messy...)
def remove_collisions(lbl_model0, idx_model0, lbl_model1, idx_model1):
    # convert tensors to np arrays, as mixing use of tensors
    # and arrays together can cause some strange behaviors
    # (they will still both point to the same place in memory)
    lbl_model0_np = lbl_model0.cpu().detach().numpy()
    lbl_model1_np = lbl_model1.cpu().detach().numpy()
    idx_model0_np = idx_model0.cpu().detach().numpy()
    idx_model1_np = idx_model1.cpu().detach().numpy()
    
    # find which instances have been labeled with 
    # the most confidence by both model0 and model1
    inter, idx_inter0, idx_inter1 = np.intersect1d(
                                        idx_model0_np,
                                        idx_model1_np,
                                        return_indices=True)

    print('Intersection: {} \nidx_inter0: {} \nidx_inter1: {}'
          .format(inter, idx_inter0, idx_inter1))

    # which instances have a conflicting prediction from both models?
    mask_coll = lbl_model0_np[idx_inter0] != lbl_model1_np[idx_inter1]
    idx_colls = inter[mask_coll]
    lbl_colls = []

    if (len(idx_colls) > 0):
        print("Idx cols", idx_colls)
        idx_coll0 = idx_inter0[mask_coll]
        idx_coll1 = idx_inter1[mask_coll]
        
        mask0 = np.ones(len(idx_model0), dtype=bool)
        mask1 = np.ones(len(idx_model1), dtype=bool)
        mask0[idx_coll0] = False
        mask1[idx_coll1] = False
        
        lbl_coll0 = lbl_model0_np[mask0]
        lbl_coll1 = lbl_model1_np[mask1]
        lbl_colls = list(zip(lbl_coll0, lbl_coll1))

        mask = mask0 & mask1
        lbl_model0_np = lbl_model0_np[mask]
        idx_model0_np = idx_model0_np[mask]

        lbl_model1_np = lbl_model1_np[mask]
        idx_model1_np = idx_model1_np[mask]

    return lbl_model0_np, idx_model0_np, lbl_model1_np, idx_model1_np, lbl_colls, idx_colls

In [5]:
torch.manual_seed(13)

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"using {device}")

using cuda


In [7]:
###################################################################
# verifying that conflicting predictions are removed accordingly

In [8]:
out0 = torch.rand((4, 3))
out0

tensor([[0.0918, 0.4794, 0.8106],
        [0.0151, 0.0153, 0.6036],
        [0.2318, 0.8633, 0.9859],
        [0.1975, 0.0830, 0.4253]])

In [9]:
probs0, labs0, idxs0 = get_topk_predictions(out0, 3)
print(probs0, labs0, idxs0)

tensor([0.9859, 0.8106, 0.6036]) tensor([2, 2, 2]) tensor([2, 0, 1])


In [10]:
labs1 = torch.tensor([1, 0, 1])
idxs1 = torch.tensor([5, 3, 4])
remove_collisions(labs0, idxs0, labs1, idxs1)

Intersection: [] 
idx_inter0: [] 
idx_inter1: []


(array([2, 2, 2]),
 array([2, 0, 1]),
 array([1, 0, 1]),
 array([5, 3, 4]),
 [],
 array([], dtype=int64))

In [11]:
labs2 = torch.tensor([1, 2, 0])
idxs2 = torch.tensor([1, 2, 0])
remove_collisions(labs0, idxs0, labs2, idxs2)

Intersection: [0 1 2] 
idx_inter0: [1 2 0] 
idx_inter1: [2 0 1]
Idx cols [0 1]


(array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 [(2, 2)],
 array([0, 1]))

In [12]:
labs3 = torch.tensor([1, 2, 2])
idxs3 = torch.tensor([2, 0, 1])
remove_collisions(labs0, idxs0, labs3, idxs3)

Intersection: [0 1 2] 
idx_inter0: [1 2 0] 
idx_inter1: [1 2 0]
Idx cols [2]


(array([2, 2]),
 array([0, 1]),
 array([2, 2]),
 array([0, 1]),
 [(2, 2), (2, 2)],
 array([2]))

In [13]:
labs4 = torch.tensor([0, 0, 0])
idxs4 = torch.tensor([2, 0, 1])
remove_collisions(labs0, idxs0, labs4, idxs4)

Intersection: [0 1 2] 
idx_inter0: [1 2 0] 
idx_inter1: [1 2 0]
Idx cols [0 1 2]


(array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 [],
 array([0, 1, 2]))

In [14]:
#####################################################################################
# verifying that function to split the dataset works as intended

In [15]:
# splits the datasets of the two views so that
# the instances inside are still aligned by index
def train_test_split_samples(samples0, samples1, test_size, random_state=None):
    if random_state is not None:
        random.seed(random_state)

    assert test_size > 0 and test_size < 1, \
        'test_size should be a float between (0, 1)'

    assert len(samples0) == len(samples1), \
        'number of samples in samples0, samples1 are not equal'
    
    idx_samples = list(range(len(samples0)))
    idx_test = random.sample(idx_samples, floor(test_size * len(samples0)))
    idx_train = list(set(idx_samples) - set(idx_test))

    # convert to np array for convenient array indexing
    samples0_np = np.stack([np.array(a) for a in samples0])
    samples1_np = np.stack([np.array(a) for a in samples1])
    
    samples_train0 = [(str(a[0]), int(a[1])) for a in list(samples0_np[idx_train])]
    samples_test0 = [(str(a[0]), int(a[1])) for a in list(samples0_np[idx_test])]
    samples_train1 = [(str(a[0]), int(a[1])) for a in list(samples1_np[idx_train])]
    samples_test1 = [(str(a[0]), int(a[1])) for a in list(samples1_np[idx_test])]

    return samples_train0, samples_test0, samples_train1, samples_test1

In [16]:
with open('cotraining_samples_lists_fixed.pkl', 'rb') as fp:
    dict = pickle.load(fp)

In [17]:
print(f"Number of samples: {len(dict['labeled'])}")

Number of samples: 4303


In [18]:
# split data into labeled/unlabeled
samples_train0, samples_unlbl0, samples_train1, samples_unlbl1 = \
    train_test_split_samples(dict['labeled'], dict['inferred'],
                             test_size=0.75, random_state=13)

In [19]:
print(f"Number of samples (train0): {len(samples_train0)}")
print(f"Number of samples (unlabeled0): {len(samples_unlbl0)}")

print(f"Number of samples (train1): {len(samples_train1)}")
print(f"Number of samples (unlabeled1): {len(samples_unlbl1)}")

Number of samples (train0): 1076
Number of samples (unlabeled0): 3227
Number of samples (train1): 1076
Number of samples (unlabeled1): 3227


In [20]:
# split the data so we get 70/10/20 train/val/test
samples_train0, samples_test0, samples_train1, samples_test1 = \
        train_test_split_samples(samples_train0, samples_train1,
                                 test_size=0.2, random_state=13)

samples_train0, samples_val0, samples_train1, samples_val1 = \
        train_test_split_samples(samples_train0, samples_train1,
                                 test_size=0.125, random_state=13)

In [21]:
print(f"Number of samples (train0): {len(samples_train0)}")
print(f"Number of samples (validation0): {len(samples_val0)}")
print(f"Number of samples (test0): {len(samples_test0)}")
print(f"Number of samples (unlabeled0): {len(samples_unlbl0)}")

print(f"Number of samples (train1): {len(samples_train1)}")
print(f"Number of samples (validation1): {len(samples_val1)}")
print(f"Number of samples (test1): {len(samples_test1)}")
print(f"Number of samples (unlabeled1): {len(samples_unlbl1)}")

Number of samples (train0): 754
Number of samples (validation0): 107
Number of samples (test0): 215
Number of samples (unlabeled0): 3227
Number of samples (train1): 754
Number of samples (validation1): 107
Number of samples (test1): 215
Number of samples (unlabeled1): 3227


In [22]:
#####################################################################################
# making some dummy imagefolder objects to pass in the samples

In [23]:
trans = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor()
    ])

In [24]:
def create_imagefolder(data, samples, path, transform=None, new_path=None):
    imgfolder = datasets.ImageFolder(path, transform=transform)
    imgfolder.class_to_idx = data['class_map']
    imgfolder.classes = list(data['class_map'].keys())
    imgfolder.samples = samples

    if new_path is not None:
        imgfolder.root = new_path

    return imgfolder

In [25]:
# Create ImageFolder objects for first view
dummy_path = '/ourdisk/hpc/ai2es/jroth/data/labeled'
data_train0 = create_imagefolder(dict, samples_train0, dummy_path, trans)
data_unlbl0 = create_imagefolder(dict, samples_unlbl0, dummy_path, trans)
data_val0 = create_imagefolder(dict, samples_val0, dummy_path, trans)
data_test0 = create_imagefolder(dict, samples_test0, dummy_path, trans)

# Create ImageFolder objects for second view (we will also update the root/path)
new_path = '/ourdisk/hpc/ai2es'
data_train1 = create_imagefolder(dict, samples_train1, dummy_path, trans, new_path)
data_unlbl1 = create_imagefolder(dict, samples_unlbl1, dummy_path, trans, new_path)
data_val1 = create_imagefolder(dict, samples_val1, dummy_path, trans, new_path)
data_test1 = create_imagefolder(dict, samples_test1, dummy_path, trans, new_path)

In [26]:
print(data_train0)
print(data_train1)

Dataset ImageFolder
    Number of datapoints: 754
    Root location: /ourdisk/hpc/ai2es/jroth/data/labeled
    StandardTransform
Transform: Compose(
               Resize(size=256, interpolation=bilinear, max_size=None, antialias=warn)
               CenterCrop(size=(224, 224))
               ToTensor()
           )
Dataset ImageFolder
    Number of datapoints: 754
    Root location: /ourdisk/hpc/ai2es
    StandardTransform
Transform: Compose(
               Resize(size=256, interpolation=bilinear, max_size=None, antialias=warn)
               CenterCrop(size=(224, 224))
               ToTensor()
           )


In [27]:
#####################################################################################
# testing the dataset updates so that they work correctly (samples added and removed)

In [28]:
def add_to_imagefolder(paths, labels, dataset):
    """
    Adds the paths with the labels to an image classification dataset

    :list paths: a list of absolute image paths to add to the dataset
    :list labels: a list of labels for each path
    :Dataset dataset: the dataset to add the samples to
    """

    new_samples = list(zip(paths, labels))

    dataset.samples += new_samples

    return dataset.samples

In [29]:
def predict(loader, model, device):
    print(f"Number of instances to predict: {len(loader.dataset)}")
    model.eval()
    predictions = []
    with torch.no_grad():
        for X, y in loader:
            X, y = X.to(device), y.to(device)
            output = model(X)
            predictions.append(output)
    return torch.cat(predictions) # output shape (# instances, # outputs)

In [30]:
def cotrain(loader0, loader1, loader_unlbl0, loader_unlbl1,
            model0, model1, k, device):
    pred_model0 = predict(loader_unlbl0, model0, device)
    pred_model1 = predict(loader_unlbl1, model1, device)

    _, lbl_topk0, idx_topk0 = get_topk_predictions(
                                    pred_model0,
                                    k if k <= len(pred_model0) else len(pred_model0))
    _, lbl_topk1, idx_topk1 = get_topk_predictions(
                                    pred_model1, 
                                    k if k <= len(pred_model1) else len(pred_model1))

    print(f"Number of unlabeled instances: {len(loader_unlbl0.dataset)}")

    # if two models predict confidently on the same instance,
    # find and remove conflicting predictions from the lists
    lbl_topk0, idx_topk0, lbl_topk1, idx_topk1, lbl_colls, idx_colls = \
    remove_collisions(lbl_topk0, idx_topk0, lbl_topk1, idx_topk1)

    print("\nLast 3 elements of top-k:\n lbl0: {}\t lbl1: {}\t idx0: {}\t idx1: {}"
          .format(lbl_topk0[-3:], lbl_topk1[-3:],
                 idx_topk0[-3:], idx_topk1[-3:]))

    samples_unlbl0 = np.stack([np.array(a) for a in loader_unlbl0.dataset.samples])
    samples_unlbl1 = np.stack([np.array(a) for a in loader_unlbl1.dataset.samples])

    if len(idx_colls) > 0:
        print("\nImage paths, labels of collisions:\n unlbl0: {}\n unlbl1: {}\n labels: {}"
             .format(samples_unlbl0[idx_colls],
                     samples_unlbl1[idx_colls], 
                     lbl_colls))

    # retrieve the instances that have been labeled with high confidence by the other model
    list_samples0 = [(str(a[0]), int(a[1])) for a in list(samples_unlbl0[idx_topk1])]
    list_samples1 = [(str(a[0]), int(a[1])) for a in list(samples_unlbl1[idx_topk0])]

    print("\nLast 3 instances:\n samples0:\n {} \n samples1:\n {}"
          .format(list_samples0[-3:], list_samples1[-3:]))

    # image paths for both
    paths0 = [i for i, _ in list_samples0]
    paths1 = [i for i, _ in list_samples1]

    # add pseudolabeled instances to the labeled datasets
    loader0.dataset.samples = add_to_imagefolder(paths0, lbl_topk1.tolist(), loader0.dataset)
    loader1.dataset.samples = add_to_imagefolder(paths1, lbl_topk0.tolist(), loader1.dataset)

    print("\nLast 3 instances:\n loader0:\n {} \n loader1:\n {}"
          .format(loader0.dataset.samples[-3:],
                  loader1.dataset.samples[-3:]))

    # remove instances from unlabeled dataset
    mask = np.ones(len(loader_unlbl0.dataset), dtype=bool)
    mask[idx_topk0] = False
    mask[idx_topk1] = False

    print(f"Number of unlabeled instances to remove: {(~mask).sum()}")

    samples_unlbl0 = samples_unlbl0[mask]
    samples_unlbl1 = samples_unlbl1[mask]

    list_unlbl0 = [(str(a[0]), int(a[1])) for a in list(samples_unlbl0)]
    list_unlbl1 = [(str(a[0]), int(a[1])) for a in list(samples_unlbl1)]

    loader_unlbl0.dataset.samples = list_unlbl0
    loader_unlbl1.dataset.samples = list_unlbl1

    print(f"Remaining number of unlabeled instances: {len(loader_unlbl0.dataset)}")

In [31]:
loader_kwargs = {'batch_size': 64, 'shuffle': False}

loader_train0 = DataLoader(data_train0, **loader_kwargs)
loader_unlbl0 = DataLoader(data_unlbl0, **loader_kwargs)
loader_val0 = DataLoader(data_val0, **loader_kwargs)
loader_test0 = DataLoader(data_test0, **loader_kwargs)

loader_train1 = DataLoader(data_train1, **loader_kwargs)
loader_unlbl1 = DataLoader(data_unlbl1, **loader_kwargs)
loader_val1 = DataLoader(data_val1, **loader_kwargs)
loader_test1 = DataLoader(data_test1, **loader_kwargs)

In [32]:
# copies to verify that things are getting removed as they should
train0_copy = copy.deepcopy(data_train0)
unlbl0_copy = copy.deepcopy(data_unlbl0)

train1_copy = copy.deepcopy(data_train1)
unlbl1_copy = copy.deepcopy(data_unlbl1)

train0_copy_np = np.stack([np.array(a) for a in train0_copy.samples])
unlbl0_copy_np = np.stack([np.array(a) for a in unlbl0_copy.samples])

train1_copy_np = np.stack([np.array(a) for a in train1_copy.samples])
unlbl1_copy_np = np.stack([np.array(a) for a in unlbl1_copy.samples])

In [33]:
model0 = resnet50(num_classes=3).to(device)

# re-set the seed so we get a different set of weights
torch.manual_seed(1729)
model1 = resnet50(num_classes=3).to(device)

In [34]:
k = int(len(loader_unlbl0.dataset) * 0.01)
print(f"k: {k}")

cotrain(loader_train0, loader_train1, loader_unlbl0, loader_unlbl1, model0, model1, k, device)

k: 32
Number of instances to predict: 3227
Number of instances to predict: 3227
Number of unlabeled instances: 3227
Intersection: [] 
idx_inter0: [] 
idx_inter1: []

Last 3 elements of top-k:
 lbl0: [2 2 2]	 lbl1: [1 1 1]	 idx0: [2866 1108  263]	 idx1: [464 472 477]

Last 3 instances:
 samples0:
 [('/ourdisk/hpc/ai2es/jroth/data/labeled/bronx_allsites/dry/NYSDOT_m4er5dez4ab_2022-01-26-05-01-16.jpg', 0), ('/ourdisk/hpc/ai2es/jroth/data/labeled/bronx_allsites/dry/NYSDOT_m4er5dez4ab_2022-01-26-11-31-06.jpg', 0), ('/ourdisk/hpc/ai2es/jroth/data/labeled/bronx_allsites/dry/NYSDOT_m4er5dez4ab_2022-02-15-23-00-47.jpg', 0)] 
 samples1:
 [('/ourdisk/hpc/ai2es/datasets/DOT/Skyline_6464/20220222/I_87_at_Interchange_3_(Yonkers_Mile_Square_Road)__Northbound__Skyline_6464_2022-02-22-22:55:27.jpg', 2), ('/ourdisk/hpc/ai2es/datasets/DOT/Skyline_6464/20220203/I_87_at_Interchange_3_(Yonkers_Mile_Square_Road)__Northbound__Skyline_6464_2022-02-03-23:45:28.jpg', 2), ('/ourdisk/hpc/ai2es/datasets/DOT/Skyline

In [35]:
# Some checks to see if the images are still aligned
# and the datasets are updated correctly (...)

print("instances in original, updated unlabeled set (view 0):")
temp_unlbl0 = np.stack([np.array(a) for a in loader_unlbl0.dataset.samples])
print(unlbl0_copy_np[[[64, 472, 477]])
print(temp_unlbl0[[464, 472, 477]])

print("\ninstances in original, updated unlabeled set (view 1):")
temp_unlbl1 = np.stack([np.array(a) for a in loader_unlbl1.dataset.samples])
print(unlbl1_copy_np[[2866, 1108, 263]])
print(temp_unlbl1[[2866, 1108, 263]])

print("\nlast 3 instances in original train set:")
print(train0_copy_np[-3:][:,0])
print(train1_copy_np[-3:][:,0])

print("\nlast 3 instances in updated train set:")
temp_train0 = np.stack([np.array(a) for a in loader_train0.dataset.samples])
temp_train1 = np.stack([np.array(a) for a in loader_train1.dataset.samples])
print(temp_train0[-3:])
print(temp_train1[-3:])

print("\n3 elements from top-k predictions:")
print(unlbl0_copy_np[[464, 472, 477]][:,0])
print(unlbl1_copy_np[[2866, 1108, 263]][:,0])

instances in original, updated unlabeled set (view 0):
[['/ourdisk/hpc/ai2es/jroth/data/labeled/bronx_allsites/dry/NYSDOT_m4er5dez4ab_2022-01-26-05-01-16.jpg'
  '0']
 ['/ourdisk/hpc/ai2es/jroth/data/labeled/bronx_allsites/dry/NYSDOT_m4er5dez4ab_2022-01-26-11-31-06.jpg'
  '0']
 ['/ourdisk/hpc/ai2es/jroth/data/labeled/bronx_allsites/dry/NYSDOT_m4er5dez4ab_2022-02-15-23-00-47.jpg'
  '0']]
[['/ourdisk/hpc/ai2es/jroth/data/labeled/bronx_allsites/wet/NYSDOT_m4er5dez4ab_2022-02-04-04-46-10.jpg'
  '2']
 ['/ourdisk/hpc/ai2es/jroth/data/labeled/bronx_allsites/snow/NYSDOT_m4er5dez4ab_2022-01-29-09-35-57.jpg'
  '1']
 ['/ourdisk/hpc/ai2es/jroth/data/labeled/bronx_allsites/dry/NYSDOT_uyomtjhwsay_2022-02-21-18-00-46.jpg'
  '0']]

instances in original, updated unlabeled set (view 1):
[['/ourdisk/hpc/ai2es/datasets/DOT/Skyline_6464/20220222/I_87_at_Interchange_3_(Yonkers_Mile_Square_Road)__Northbound__Skyline_6464_2022-02-22-22:55:27.jpg'
  '2']
 ['/ourdisk/hpc/ai2es/datasets/DOT/Skyline_6464/20220203

In [36]:
#####################################################################################

In [37]:
# def train(loader, model, loss_fn, optimizer, device):
#     size = len(loader.dataset)
#     model.train()
#     for batch, (X, y) in enumerate(loader):
#         X, y = X.to(device), y.to(device)
#         optimizer.zero_grad()
        
#         # Compute prediction error
#         pred = model(X)
#         loss = loss_fn(pred, y)

#         # Backpropagation
#         loss.backward()
#         optimizer.step()
        
#         loss, current = loss, (batch + 1) * len(X)
#         print(f"loss: {loss:>7f} [{current:5d} / {size:>5d}]")

In [38]:
# def test(loader, model, loss_fn, device):
#   size = len(loader.dataset)
#   num_batches = len(loader)
#   model.eval()
#   test_loss, correct = 0, 0
#   with torch.no_grad():
#     for X, y in loader:
#       X, y = X.to(device), y.to(device)
#       pred = model(X)
#       test_loss += loss_fn(pred, y).item()
#       correct += (pred.argmax(1) == y).type(torch.float).sum().item()
#     test_loss /= num_batches
#     correct /= size
      
#     print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f}")
#     return correct, test_loss

In [39]:
# # define loss function and optimizer
# loss_fn = nn.CrossEntropyLoss()
# optimizer0 = torch.optim.SGD(model0.parameters(), lr=1e-3,momentum=0.9)
# optimizer1 = torch.optim.SGD(model1.parameters(), lr=1e-3, momentum=0.9)

# # we also need to define some sort of learning rate/early stopping scheduler
# scheduler0 = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer0)
# scheduler1 = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer1)

In [40]:
# # re-update the train, unlabeled datasets as those were modified previously
# loader_train0 = DataLoader(train0_copy, **loader_kwargs)
# loader_unlbl0 = DataLoader(unlbl0_copy, **loader_kwargs)

# loader_train1 = DataLoader(train1_copy, **loader_kwargs)
# loader_unlbl1 = DataLoader(unlbl1_copy, **loader_kwargs)

In [41]:
# print(len(loader_train0.dataset))
# print(len(loader_unlbl0.dataset))

# print(len(loader_train1.dataset))
# print(len(loader_unlbl1.dataset))

In [42]:
# iterations = 1
# epochs = 3
# k = int(len(loader_unlbl0.dataset) * 0.05)
# print(f"k: {k}")

# for i in range(iterations):
#     print(f"Co-training iteration {i + 1}")
#     print("training model0...")
#     for epoch in range(epochs):
#         print(f"Epoch {epoch + 1} \n-------------------------------------------")
#         train(loader_train0, model0, loss_fn, optimizer0, device)
#         val_acc0, val_loss0 = test(loader_val0, model0, loss_fn, device)
#         # NOTE should have an early stopping check here
#         scheduler0.step(val_loss0)

#     print("training model1...")
#     for epoch in range(epochs):
#         print(f"Epoch {epoch + 1} \n-------------------------------------------")
#         train(loader_train1, model1, loss_fn, optimizer1, device)
#         val_acc1, val_loss1 = test(loader_val1, model1, loss_fn, device)
#         # NOTE should have an early stopping check here
#         scheduler0.step(val_loss1)

#     cotrain(loader_train0, loader_train1, loader_unlbl0, loader_unlbl1, model0, model1, k, device)
#     test_acc0, test_loss0 = test(loader_test0, model0, loss_fn, device)
#     test_acc1, test_loss1 = test(loader_test1, model1, loss_fn, device)